In [ ]:
import pandas as pd
import numpy as np
import itertools
import random
import ast
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, wilcoxon, friedmanchisquare, kendalltau, pearsonr
from matplotlib import animation
from mpl_toolkits import mplot3d
from scipy.spatial import distance
from pyxdameraulevenshtein import damerau_levenshtein_distance

pd.set_option('display.max_rows', 500)

### Prepare Data

In [ ]:
# read data

df = pd.read_csv('all_task_environments_new.csv', header=0, skiprows=[1])

### Define functions for prediction

In [ ]:
def predict_sequence(objects, coordinates, start_coordinates, c, k, dimension=[3,]):
    ''' Predicts sequence based on required objects, object coordinates, start coordinates of subject,
        parameters (c+k) and dimensionality.
        Input: Objects, object coordinates, start coordinates, c, k, dimension
        Output: Sequence of objects as str
    '''
    prediction = []
    possible_items = dict.fromkeys(objects, 0) # generate dict from object list
    coord_index = 0
    start_coords = start_coordinates
    coords = coordinates
    new_coords = {}
    new_start_coords = []
    
    if dimension[0] == 3: # no changes if 3D
        new_coords = coords
        new_start_coords = start_coords
        
    elif dimension[0] == 2: # 2D: remove obsolete coordinate
        if dimension[1] == 'xy':
            new_coords = {key: value[:-1] for key, value in coords.items()}
            new_start_coords = [x[:-1] for x in start_coords]
                
        elif dimension[1] == 'xz':
            new_start_coords = [[x[0], x[-1]] for x in start_coords]
            
            for key, value in coords.items():
                new_value = (value[0], value[-1])
                new_coords[key] = new_value
        
        elif dimension[1] == 'yz':
            new_coords = {key: value[1:] for key, value in coords.items()}
            new_start_coords = [x[1:] for x in start_coords]
                
    elif dimension[0] == 1: # 1D: choose appropriate coordinate
        if dimension[1] == 'x':
            new_coords = {key: value[0] for key, value in coords.items()}
            new_start_coords = [x[0] for x in start_coords]
        
        elif dimension[1] == 'y':
            new_coords = {key: value[1] for key, value in coords.items()}
            new_start_coords = [x[1] for x in start_coords]
        
        elif dimension[1] == 'z':
            new_coords = {key: value[2] for key, value in coords.items()}
            new_start_coords = [x[2] for x in start_coords]
    
    while bool(possible_items) == True: # while dict not empty
        for obj in possible_items.keys():
            possible_items[obj] = ((distance.euclidean(
                                new_start_coords[coord_index], 
                                new_coords[obj])
                                ) ** k[obj]) * c[obj]
        #print(possible_items)                      
        minval = min(possible_items.values())
        minval = [k for k, v in possible_items.items() if v == minval]
        minval = random.choice(minval) # choose prediction randomly if multiple items have same cost
        prediction.append(minval)
        del possible_items[minval]
        coord_index += 1
        
    return prediction

In [ ]:
def get_average(objects, coordinates, start_coordinates, c, k, dimension, sequence):
    ''' Returns average edit distance (Damerau-Levenshtein) for 100 trials of sequence prediction.
    '''
    edit_list = []

    for x in range(0,100):
        result = ''.join(predict_sequence(objects, coordinates, start_coordinates, c, k, dimension))
        dl = damerau_levenshtein_distance(sequence,result)
        edit_list.append(dl)
    
    avg = np.mean(edit_list)
    median = np.median(edit_list)
    return avg, median

In [ ]:
def get_avg_editdist(data):
    ''' Calculates average edit distance for all combinations of parameters (c, k, dimension).
        Input: Dataframe with objects, coordinates, start coordinates, object categories
        Output: Dataframe with edit distance results (col name: parameters used)
    '''
    results = pd.DataFrame()
    dimensions = [[1,'x'],[1,'y'],[1,'z'],[2,'xy'],[2,'xz'],[2,'yz'],[3,'xyz']]
        
    for row in range(0,len(df)):
        objects = list(df.at[row,'objects'].split(','))
        strong_k = list(df.at[row,'strong_k'].split(','))
        mid_k = list(df.at[row,'mid_k'].split(','))
        coordinates = {key: ast.literal_eval(value) for key, value in (elem.split(': ') for elem in df.at[row,'coordinates'].split(';'))}
        start_coordinates = list(ast.literal_eval(df.at[row, 'start_coordinates']))
        sequence = str(df.at[row,'sequence'])
        
        #for k in np.arange(0.0,1.0,0.1):
        for k in np.arange(0,0.9,0.1):
            k_strong = round(k,2)
            k_mid = round(k + 0.1,2)
            k1 = {obj: k_strong if obj in strong_k else k_mid if obj in mid_k else 1.0 for obj in objects}
                    
            for c in np.arange(1.0,2.0,0.1):
                c = round(c, 1)
                c1 = {obj: c if obj in df.at[row, 'containment'] else 1.0 for obj in objects}
                
                for dim in dimensions:                
                    # get average edit distance
                    edit_dist, median = get_average(objects, coordinates, start_coordinates, c1, k1, dim, sequence)
                    edit_dist = edit_dist / len(sequence)
                    median = median / len(sequence)
                    
                    params = 'c: ' + str(c) + '; k: ' + str(k_strong) + ',' + str(k_mid) + '; ' + str(dim[1])
                    results.at[row,params] = median
                    
    return results

### Calculate edit distances, create df

In [ ]:
#%%timeit -n1 -r1
results_new = get_avg_editdist(df)

In [ ]:
results_new

### Get lowest error, compare edit distances

In [ ]:
def get_lowest_error(results):
    ''' Returns lowest error in dataframe and index of lowest error.
    '''
    for col in list(results):
        results.loc['mean',col] = results[col].mean()
        results.loc['median',col] = results[col].median()
    lowest = min(results.loc['mean'])
    mean = list(results.loc['mean'])
    
    return lowest, results.columns[(results.loc['mean'] == lowest)], mean, results

In [ ]:
lowest, lowest_idx, list_mean, results_mean = get_lowest_error(results_new)
lowest, lowest_idx

In [ ]:
mean_editdist = np.mean(list_mean)
median_editdist = np.median(results_mean.loc['median',:])
print(mean_editdist, median_editdist)

In [ ]:
# Compare avg edit distance for xy and xyz
list_xy = []
list_xyz = []
median_xy = []
median_xyz = []

for col in results_mean:
    if col.split(';')[2].strip() == 'xy':
        list_xy.append(results_mean.at['mean',col])
        median_xy.append(results_mean.at['median',col])
    elif col.split(';')[2].strip() == 'xyz':
        list_xyz.append(results_mean.at['mean',col])
        median_xyz.append(results_mean.at['median',col])
        
avg_xy = np.mean(list_xy)
med_xy = np.median(median_xy)
std_xy = np.std(list_xy)
print('Average xy: ' + str(avg_xy) + ', median: ' + str(med_xy)  + ', stdev: ' + str(std_xy))
        
avg_xyz = np.mean(list_xyz)
med_xyz = np.median(median_xyz)
std_xyz = np.std(list_xyz)
print('Average xyz: ' + str(avg_xyz) + ', median: ' + str(med_xyz) + ', stdev: ' + str(std_xyz))

In [ ]:
len(list_xy)

In [ ]:
# Compare avg edit distances for x, y, z
list_x = []
list_y = []
list_z = []
list_xz = []
list_yz = []

for col in results_mean:
    if col.split(';')[2].strip() == 'x':
        list_x.append(results_mean.at['mean',col])
    elif col.split(';')[2].strip() == 'y':
        list_y.append(results_mean.at['mean',col])
    elif col.split(';')[2].strip() == 'z':
        list_z.append(results_mean.at['mean',col])
    elif col.split(';')[2].strip() == 'xz':
        list_xz.append(results_mean.at['mean',col])
    elif col.split(';')[2].strip() == 'yz':
        list_yz.append(results_mean.at['mean',col])

avg_xz = np.mean(list_xz)
avg_yz = np.mean(list_yz)
        
avg_x = np.mean(list_x)
lowest_x = min(list_x)
print('Average x: ' + str(avg_x) + ', min: ' + str(lowest_x)) 
        
avg_y = np.mean(list_y)
lowest_y = min(list_y)
print('Average y: ' + str(avg_y) + ', min: ' + str(lowest_y)) 
        
avg_z = np.mean(list_z)
lowest_z = min(list_z)
print('Average z: ' + str(avg_z) + ', min: ' + str(lowest_z)) 

print('Average xz: ' + str(avg_xz)) 
print('Average yz: ' + str(avg_yz)) 

### Statistical analysis

In [ ]:
# correlate avg edit distance with volume of environment space

dimensions_error = [avg_x, avg_y, avg_z, avg_xy, avg_xz, avg_yz, avg_xyz]
dimensions_volume = [1.889,3.1667,1.8333,6.5556,3.5,6.0556,12.5556]
dimensions_volume_corr = [1.889,3.1667,1.8333,6.5556,3.5,6.0556,19.1111]
#volumes_z 

corr, p = spearmanr(dimensions_error, dimensions_volume)
print('Spearman dims: \u03C1 = %.3f, p = %.15f' % (corr, p))

means = list(results_mean.loc['mean',:])
dimensions_vol = dimensions_volume * 90
dimensions_vol_corr = dimensions_volume_corr * 90

corr, p = spearmanr(means, dimensions_vol)
print('Spearman dims: \u03C1 = %.3f, p = %.20f' % (corr, p))

corr, p = spearmanr(means, dimensions_vol_corr)
print('Spearman dims corrected: \u03C1 = %.3f, p = %.20f' % (corr, p))

In [ ]:
# Compare all error predictions for dimensions
stat, p = friedmanchisquare(list_x,list_y,list_z,list_xy,list_xz,list_yz,list_xyz)
print('Friedman: stat = %.3f, p = % 10.3E' % (stat, p))

In [ ]:
# compare individual dimensions (error predictions)

# 2D
print('2D:')
corr, p = spearmanr(list_x, list_xy)
print('Spearman x: \u03C1 = %.3f, p = %.15f' % (corr, p))

corr, p = spearmanr(list_y, list_xy)
print('Spearman y: \u03C1 = %.3f, p = %.15f' % (corr, p))

corr, p = kendalltau(list_y, list_xy)
print('Kendall: \u03C4 b = %.3f, p = %.15f' % (corr, p))

# 3D
print('3D:')
corr, p = spearmanr(list_x, list_xyz)
print('Spearman x: \u03C1 = %.3f, p = %.15f' % (corr, p))

corr, p = spearmanr(list_y, list_xyz)
print('Spearman y: \u03C1 = %.3f, p = %.15f' % (corr, p))

corr, p = spearmanr(list_z, list_xy)
print('Spearman z: \u03C1 = %.3f, p = %.15f' % (corr, p))

# 2D vs 3D
print('2D vs 3D:')
stat, p = wilcoxon(list_xy, list_xyz, zero_method='wilcox')
print('Wilcoxon: W = %.3f, p = %.3f' % (stat, p))

stat, p = wilcoxon(list_xy, list_xyz, zero_method='wilcox', alternative='less')
print('Wilcoxon: W = %.3f, p = %.3f' % (stat, p))

In [ ]:
# 2D vs 3D median
print('2D vs 3D medians:')
stat, p = wilcoxon(median_xy, median_xyz, zero_method='wilcox')
print('Wilcoxon: W = %.3f, p = %.3f' % (stat, p))

### Prepare data + plots

In [ ]:
# prepare data for plots

means_x, means_y, means_z, means_xy, means_xz, means_yz, means_xyz = ([] for i in range(7))

for i in range(0,len(means),7):
    means_x.append(means[i])
    means_y.append(means[i+1])
    means_z.append(means[i+2])
    means_xy.append(means[i+3])
    means_xz.append(means[i+4])
    means_yz.append(means[i+5])
    means_xyz.append(means[i+6])
    
dim_x = []
dim_x.append(dimensions_volume[0])
dim_x = dim_x * 90

dim_y = []
dim_y.append(dimensions_volume[1])
dim_y = dim_y * 90

dim_z = []
dim_z.append(dimensions_volume[2])
dim_z = dim_z * 90

dim_xy = []
dim_xy.append(dimensions_volume[3])
dim_xy = dim_xy * 90

dim_xz = []
dim_xz.append(dimensions_volume[4])
dim_xz = dim_xz * 90

dim_yz = []
dim_yz.append(dimensions_volume[5])
dim_yz = dim_yz * 90

dim_xyz = []
dim_xyz.append(dimensions_volume[6])
dim_xyz = dim_xyz * 90

In [ ]:
# plot error rate for all dims depending on volume

cm = ['red','blue','green','magenta','cyan','orange','grey']

fig = plt.figure(figsize=(12,8))
plot_x = plt.scatter(means_x, dim_x, alpha=0.5, c=cm[0], label='x')
plot_y = plt.scatter(means_y, dim_y, alpha=0.5, c=cm[1], label='y')
plot_z = plt.scatter(means_z, dim_z, alpha=0.5, c=cm[2], label='z')
plot_xy = plt.scatter(means_xy, dim_xy, alpha=0.5, c=cm[3], label='xy')
plot_xz = plt.scatter(means_xz, dim_xz, alpha=0.5, c=cm[4], label='xz')
plot_yz = plt.scatter(means_yz, dim_yz, alpha=0.5, c=cm[5], label='yz')
plot_xyz = plt.scatter(means_xyz, dim_xyz, alpha=0.5, c=cm[6], label='xyz')

plt.plot(np.unique(means), np.poly1d(np.polyfit(means, dimensions_vol, 1))(np.unique(means)), color='black')
plt.ylabel('volume of environment', fontsize=25, labelpad=10)
plt.xlabel('avg. edit distance', fontsize=25, labelpad=10)

# create legend
plt.legend(scatterpoints=4, fontsize=18)

plt.show()

In [ ]:
# get individual points for correlation plot
editdist_x, editdist_y, editdist_z, editdist_xy, editdist_xz, editdist_yz, editdist_xyz = ([] for i in range(7))

for col in results_mean:
    if col.split(';')[2].strip() == 'x':
        editdist_x.append(list(results_mean[col].iloc[:-1]))
    elif col.split(';')[2].strip() == 'y':
        editdist_y.append(list(results_mean[col].iloc[:-1]))
    elif col.split(';')[2].strip() == 'z':
        editdist_z.append(list(results_mean[col].iloc[:-1]))
    elif col.split(';')[2].strip() == 'xy':
        editdist_xy.append(list(results_mean[col].iloc[:-1]))
    elif col.split(';')[2].strip() == 'xz':
        editdist_xz.append(list(results_mean[col].iloc[:-1]))
    elif col.split(';')[2].strip() == 'yz':
        editdist_yz.append(list(results_mean[col].iloc[:-1]))
    elif col.split(';')[2].strip() == 'xyz':
        editdist_xyz.append(list(results_mean[col].iloc[:-1]))
        
editdist_x = [item for sublist in editdist_x for item in sublist]
editdist_y = [item for sublist in editdist_y for item in sublist]
editdist_z = [item for sublist in editdist_z for item in sublist]
editdist_xy = [item for sublist in editdist_xy for item in sublist]
editdist_xz = [item for sublist in editdist_xz for item in sublist]
editdist_yz = [item for sublist in editdist_yz for item in sublist]
editdist_xyz = [item for sublist in editdist_xyz for item in sublist]

volumes_x = [3,3,3,3,2,4,2,4,1,2,2,2,2,2,1,1,1,1,1,1,2,2,2,2]
volumes_y = [3,3,3,3,3,5,4,5,2,3,3,3,3,4,2,3,3,3,3,3,2,2,2,2]
volumes_z = [2,2,2,2,2,2,2,2,0,2,3,1,1,2,1,3,3,0,3,3,1,1,1,1]
volumes_xy = [9,9,9,9,6,20,8,20,2,6,6,6,6,8,2,3,3,3,3,3,4,4,4,4]
volumes_xz = [6,6,6,6,4,8,4,8,1,4,6,2,2,4,1,3,3,1,3,3,2,2,2,2]
volumes_yz = [6,6,6,6,6,10,8,10,2,6,9,3,3,8,2,9,9,3,9,9,2,2,2,2]
volumes_xyz = [18,18,18,18,12,40,16,40,2,12,18,6,6,16,2,9,9,3,9,9,4,4,4,4]

volumes_x = volumes_x * 90
volumes_y = volumes_y * 90
volumes_z = volumes_z * 90
volumes_xy = volumes_xy * 90
volumes_xz = volumes_xz * 90
volumes_yz = volumes_yz * 90
volumes_xyz = volumes_xyz * 90

In [ ]:
# plot error rate for individual values

cm = ['red','blue','green','magenta','cyan','orange','grey']

fig = plt.figure(figsize=(12,8))
plot_x = plt.scatter(editdist_x, volumes_x, alpha=0.5, c=cm[0], label='x')
plot_y = plt.scatter(editdist_y, volumes_y, alpha=0.5, c=cm[1], label='y')
plot_z = plt.scatter(editdist_z, volumes_z, alpha=0.5, c=cm[2], label='z')
plot_xy = plt.scatter(editdist_xy, volumes_xy, alpha=0.5, c=cm[3], label='xy')
plot_xz = plt.scatter(editdist_xz, volumes_xz, alpha=0.5, c=cm[4], label='xz')
plot_yz = plt.scatter(editdist_yz, volumes_yz, alpha=0.5, c=cm[5], label='yz')
plot_xyz = plt.scatter(editdist_xyz, volumes_xyz, alpha=0.5, c=cm[6], label='xyz')

plt.plot(np.unique(means), np.poly1d(np.polyfit(means, dimensions_vol, 1))(np.unique(means)), color='black')
plt.ylabel('volume of environment', fontsize=25, labelpad=10)
plt.xlabel('avg. edit distance', fontsize=25, labelpad=10)

# create legend
plt.legend(scatterpoints=4, fontsize=18)

plt.show()

### Plots for dataframe

In [ ]:
# Define input for plots

c = [float(x[3:6]) for x in results_new.columns.tolist()]
k = [float(x[11:14]) for x in results_new.columns.tolist()]
dim = [x.strip() for x in results_new.columns.str.split(';').str[2]]
error = results_mean.loc['mean'].tolist()
median = results_mean.loc['median'].tolist()

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = plt.axes(projection='3d')

ax.plot_trisurf(c, k, error, linewidth=0.2)
ax.set_ylabel('k', fontsize=14, labelpad=7)
ax.set_xlabel('c', fontsize=14, labelpad=7)
plt.title('Average edit distance', fontsize=16)
plt.show()

In [ ]:
%matplotlib qt

cm = ['red','blue','green','magenta','cyan','orange','grey']
dim_num = [0 if x=='x' else 1 if x=='y' else 2 if x=='z' else 3 if x=='xy' else 4 if x=='xz' else 5 if x=='yz' else 6 for x in dim]

cmap = matplotlib.colors.ListedColormap(cm)

ticks = ['x', 'y', 'z', 'xy', 'xz', 'yz', 'xyz']
norm = matplotlib.colors.BoundaryNorm(ticks, cmap.N)

# create figure, 3d grid, set background to white
fig2 = plt.figure(figsize=(12,8))
ax2 = fig2.add_subplot(111, projection='3d')
ax2.w_xaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_yaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_zaxis.set_pane_color((1.0,1.0,1.0,1.0))

# create plot w/ median edit distance
img = ax2.scatter(c, k, median, alpha=0.5, s=38, c=dim_num, cmap=cmap)

# plot horizontal plane for baseline
xx, yy = np.meshgrid(np.linspace(1,1.9), np.linspace(0,0.9))
#yy = np.meshgrid(range(2), range(2))
zz = xx * 0 + 0.603
ax2.plot_surface(xx, yy, zz, alpha=0.5)

# set labels
ax2.set_ylabel('parameter k', fontsize=20, labelpad=10)
ax2.set_xlabel('parameter c', fontsize=20, labelpad=10)
ax2.set_zlabel('normalized edit distance', fontsize=20, labelpad=7)
#plt.title('Average edit distance (1 step)', fontsize=24)

# create colorbar
cb = plt.colorbar(img, cax = fig2.add_axes([0.9,0.3,0.03,0.4]))
cb.ax.set_yticklabels(ticks, fontsize=16)
plt.show()

In [ ]:
# generate rotating image

fig3 = plt.figure(figsize=(12,8))
ax3 = mplot3d.Axes3D(fig3)
ax3.w_xaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax3.w_yaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax3.w_zaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax3.set_ylabel('parameter k', fontsize=14, labelpad=7)
ax3.set_xlabel('parameter c', fontsize=14, labelpad=7)
ax3.set_zlabel('normalized edit distance', fontsize=14, labelpad=7)

def init():
    ax3.scatter(c, k, error, alpha=0.5, s=38, c=dim_num, cmap=cmap)
    
    return fig,

def animate(i):
    ax3.view_init(elev=10, azim=i*4)
    return fig,

animated = animation.FuncAnimation(fig3, animate, init_func=init, frames=90, interval=50, blit=True)

In [ ]:
# save animated fig

plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg'

fig_name = 'rotate_model_lowesterror_3D'
#animated.save(fig_name+'.gif', writer='imagemagick', fps=1000/50)

Writer = animation.writers['ffmpeg']
writer = Writer(fps=5, bitrate=10000)
animated.save(fig_name+'.mp4', writer=writer)